# Ex2

In [7]:
#from sage.all import *
import os
import hashlib

Classe fornecida para representar pontos *edward*

In [8]:
class EdPnt(object):
    def __init__(self,pt=None,curve=None,x=None,y=None):
        if pt != None:
            self.curve = pt.curve
            self.x = pt.x
            self.y = pt.y
            self.w = pt.w
        else:
            self.curve = curve
            self.x = x
            self.y = y
            self.w = x * y
    
    def eq(self, other):
        return self.x == other.x and self.y == other.y
    
    def copy(self):
        return EdPnt(curve=self.curve, x=self.x, y=self.y)
    
    def zero(self):
        return EdPnt(curve=self.curve ,x=0, y=1)
    
    def sim(self):
        return EdPnt(curve=self.curve, x= -self.x, y= self.y)
    
    def soma(self, other):
        a = self.curve.a
        d = self.curve.d
        delta = d * self.w * other.w

        self.x,self.y = (self.x * other.y + self.y * other.x)/(1 + delta),(self.y * other.y - a * self.x * other.x)/(1 - delta)
        self.w = self.x * self.y
        
    def duplica(self):
        a = self.curve.a
        d = self.curve.d
        delta = d*(self.w)**2

        self.x,self.y = (2 * self.w)/(1 + delta),(self.y**2 - a * self.x**2)/(1 - delta)
        self.w = self.x * self.y
        
    def mult(self, n):
        m = Mod(n, self.curve.L).lift().digits(2)
        Q = self.copy()
        A = self.zero()

        for b in m:
            if b == 1:
                A.soma(Q)
            Q.duplica()

        return A

Funções auxiliares, como compressão de pontos *edwards*

In [9]:
def sha512(x):
    return hashlib.sha512(str(x).encode()).digest()

def sha512_modq(s): 
    return int.from_bytes(sha512(s), "little") % ZZ(2**252 + 27742317777372353535851937790883648493)

def modp_inv(x, p): 
    return pow(x, p-2, p)

def recover_x(y, sign, p, d):
    if y >= p: 
        return None
    
    x2 = (y*y-1) * modp_inv(d*y*y+1, p)

    if x2 == 0:
        if sign:
             return None
        else: 
            return 0
        
    x = pow(x2, (p+3) // 8, p)
    x = int(x.lift())
    
    if (x*x - x2) % p != 0:
        x = x * pow(2, (p-1) // 4, p) % p 

    if (x*x - x2) % p != 0:
        return None
    
    if (int(x) & 1) != sign:
        x = p - x

    return x

def point_decompress(s,p,d):
    if len(s) != 32:
        raise Exception("Invalid input length for decompression") 
    
    y = int.from_bytes(s, "little")
    sign = y >> 255
    y &= (1 << 255) - 1

    x = recover_x(y, sign, p, d)

    if x is None:
         return None
    else:
        return (x, y, 1, x*y % p)

def point_compress(P, p):
    p_x = P.x
    p_y = P.y
    p_z = 1

    zinv = modp_inv(p_z,p)
    x = p_x * zinv % p
    y = p_y * zinv % p

    return int.to_bytes(int(int(p_y) | ((int(p_x) & 1) << 255)), 32, "little")

Classe que implementa o EdCDSA fazendo uso de *twisted edwards curves* - *edwards25519* 

In [10]:
class EdCDSA:
    def __init__(self,p,b,n,K,a,d,Px,Py,L,c):
        self.a = a
        self.d = d
        self.p = p
        self.K = K
        self.L = L
        
        self.Px = Px
        self.Py = Py

        assert a != d and is_prime(p) and p > 3
        
        A = 2 * (a + d)/(a - d)
        B = 4/(a - d)
        self.alfa = A/(3*B)
        self.s = B

        a4 = self.s**(-2) - 3*self.alfa**2
        a6 = (-self.alfa)**3 - a4*self.alfa
 
        self.EC = EllipticCurve(K, [a4, a6])
        self.P = self.ed2ec(Px,Py)

    #Calculate private key
    def private_key(self, seed):
        if (len(seed) != 32):
            raise Exception("Bad seed length")
        
        hashed_seed = sha512(seed)
        a = int.from_bytes(hashed_seed[:32], "little")
        a &= (1 << 254) - 8
        a |= (1 << 254)

        return (a, hashed_seed[32:])
        
    #Calculate public key
    def public_key(self, seed):
        (a,dummy) = self.private_key(seed)

        point = EdPnt(curve=self, x=self.Px, y=self.Py)
        point = point.mult(a)

        return point_compress(point, self.p)

    #Sign message
    def sign(self, message, seed):
        (a,prefix) = self.private_key(seed) 
        #Public key calculation
        point_A = EdPnt(curve=self, x=self.Px, y=self.Py)
        point_A = point_A.mult(a)
        A = point_compress(point_A, self.p)
        #Secret number calculation
        r = sha512_modq(prefix + message)

        #Ponto R
        point_R = EdPnt(curve=self, x=self.Px, y=self.Py)
        point_R = point_R.mult(r)
        Rs = point_compress(point_R, self.p)

        h = sha512_modq(Rs + A + message)
        s = (r + h * a) % self.L

        return Rs + int.to_bytes(int(s), 32, "little")

    #Verify message signature
    def verify(self, message, signature, public_key):
        if len(public_key) != 32: 
            raise Exception("Bad public key length") 
        if len(signature) != 64: 
            Exception("Bad signature length") 
        
        #Obter ponto representativo da public key
        A = point_decompress(public_key, self.p, self.d)
        if not A: 
            return False 
        
        #Obter ponto r
        Rs = signature[:32]
        R = point_decompress(Rs, self.p, self.d) 
        point_r = EdPnt(curve=self, x=R[0], y=R[1])

        if not R:
            return False 
        
        s = int.from_bytes(signature[32:], "little") 
        if s >= self.L: 
            return False 
        
        h = sha512_modq(Rs + public_key + message) 

        point_s = EdPnt(curve=self, x=self.Px, y=self.Py)
        point_s = point_s.mult(s)
        
        point_h = EdPnt(curve=self, x=A[0], y=A[1])
        point_h = point_h.mult(h)
        
        point_r.soma(point_h)

        #Compara os pontos
        return point_s.eq(point_r)
    
    def ed2ec(self,x,y):
        if (x,y) == (0,1):
            return self.EC(0)
        
        z = (1 + y)/(1 - y) 
        w = z/x
        
        return self.EC(z/self.s + self.alfa , w/self.s)
    
    def ec2ed(self,P):
        if P == self.EC(0):
            return (0,1)
        
        x,y = P.xy()
        u = self.s * (x - self.alfa)
        v = self.s * y
        
        return (u / v, (u - 1) / (u + 1))

## Edwards25519

Definição de todas as variáveis de acordo com o RFC 8032.

In [11]:
p = 2**255 - 19
b = 256
n = 254

K = GF(p)
a = K(-1)
d = K(-121665)/K(121666)
Px = K(15112221349535400772501151409588531511454012693041857206046113283949847762202)
Py = K(46316835694926478169428394003475163141307993866256225615783033603165251855960)

L = ZZ(2**252 + 27742317777372353535851937790883648493)
c = 3

### Teste 1 - Sucedido

In [12]:
#Gerar chave privada
private_key = os.urandom(32)

#Criar objeto
E = EdCDSA(p,b,n,K,a,d,Px,Py,L,c)
message = b"mensagem muito importante que vamos assinar"

#Calcular chave publica
public_key = E.public_key(private_key)
print("Public key: ", public_key)

#Assinar mensagem
signature = E.sign(message, private_key)
print("Signature: ", signature)

#Verificar que assinatura corresponde à mensagem
print("Verify: ", E.verify(message, signature, public_key))

Public key:  b'\n\xa1QAx\xcd\xa4\xeb\xabs\x80j\x173\xe5\xc7\xd6\xe5\x0ec%\x08\x01\x9dM\x1b\xa6P<II\x9d'
Signature:  b"\x16K\x00\xa5\xad\xbb\xff/\x12\x18uz\x1d3\xfe \xa5Ut\xf3\xa1'\xa6\n8t\xea\xd1\x7f\xd6\x9fQuzK<\xa3\xb9q\x96ySc\xb6\x1f\xcf\xf4Blb'\xdb\xf5&\xd2)\x8e{d\x1b\x88\xb6\x12\n"
Verify:  True


### Teste 2 - Não sucedido

In [13]:
#Gerar chave privada
private_key = os.urandom(32)

#Criar objeto
E = EdCDSA(p,b,n,K,a,d,Px,Py,L,c)
message = b"mensagem muito importante que vamos assinar"

#Calcular chave publica
public_key = E.public_key(private_key)
print("Public key: ", public_key)

#Assinar mensagem
signature = E.sign(message, private_key)
print("Signature: ", signature)

#Modificar mensagem
message = b"mensagem muito importante que iriamos assinar"

#Verificar que assinatura corresponde à mensagem
print("Verify: ", E.verify(message, signature, public_key))

Public key:  b'\xa392{c\x92\xbff$\\\xa8\x0ep\x8d\x02>@\xd8\x8d\xa3e\xccpJ\xdb\xafKB6\x10F\x10'
Signature:  b'k\x82\xe1\xb0\x99{\xcds3\xf3\xf4Q\x88\xb3\x01\x9c*\xf0[[\x0ex\x97\xaf\xf6J\xd3(\x01\xf3Pg\x92M\xd3H\x99LC\xcc\xf08\xcf\x92\x1f\x19\xf3\xe17&s\xa9v\x08Au\x9e\xdb\xcb\xda\xe5\xdc\x98\x0c'
Verify:  False


## C)

In [14]:
E_A = EdCDSA(p,b,n,K,a,d,Px,Py,L,c)
E_B = EdCDSA(p,b,n,K,a,d,Px,Py,L,c)

private_key = os.urandom(32)


#ALICE desafia Bob a provar que conhece a sua chave
challenge_alice = os.urandom(32)

#BOB assina o desafio para provar que conhece a chave privada
public_key_bob = E.public_key(private_key)
signature_bob = E.sign(challenge_alice, private_key)
#BOB envia assinatura e chave publica para ALICE

#ALICE verifica o desafio
verify = E_A.verify(challenge_alice, signature_bob, public_key_bob)

#Se a verificação for verdadeira, então ALICE sabe que BOB conhece a chave privada
if(verify):
    print("Bob knows the private key")
else:
    print("Bob does not know the private key")

Bob knows the private key
